<a href="https://colab.research.google.com/github/vvonchain/ML-Notebooks/blob/main/DSPy_basic_example_HuggingFaceInference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%load_ext autoreload
%autoreload 2

import sys
import os

try: # When on google Colab, let's clone the notebook so we download the cache.
    import google.colab
    repo_path = 'dspy'
    !git -C $repo_path pull origin || git clone https://github.com/stanfordnlp/dspy $repo_path
except:
    repo_path = '.'

if repo_path not in sys.path:
    sys.path.append(repo_path)

# Set up the cache for this notebook
os.environ["DSP_NOTEBOOK_CACHEDIR"] = os.path.join(repo_path, 'cache')

import pkg_resources # Install the package if it's not installed
if not "dspy-ai" in {pkg.key for pkg in pkg_resources.working_set}:
    !pip install -aU pip
    !pip install dspy-ai -qU

fatal: cannot change to 'dspy': No such file or directory
Cloning into 'dspy'...
remote: Enumerating objects: 25890, done.
remote: Counting objects: 100% (2968/2968), done.
remote: Compressing objects: 100% (857/857), done.
remote: Total 25890 (delta 2361), reused 2343 (delta 2093), pack-reused 22922
Receiving objects: 100% (25890/25890), 25.63 MiB | 18.92 MiB/s, done.
Resolving deltas: 100% (10792/10792), done.

Usage:   
  pip3 install [options] <requirement specifier> [package-index-options] ...
  pip3 install [options] -r <requirements file> [package-index-options] ...
  pip3 install [options] [-e] <vcs project url> ...
  pip3 install [options] [-e] <local project path> ...
  pip3 install [options] <archive url/path> ...

no such option: -a
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.2/182.2 kB 3.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.4/227.4 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 5.5 MB/s eta 0:00:00
 

In [ ]:
from dsp import LM
import dspy

import requests
import os

In [ ]:
class HuggingFaceInferenceClient(LM):
    def __init__(self, model, api_key=None):
        self.model = model
        self.api_key = api_key
        self.provider = "default"
        self.base_url = f"https://api-inference.huggingface.co/models/{model}"
        self.history = []

    def basic_request(self, prompt: str, **kwargs):
        headers = {
            "Content-Type": "application/json",
        }

        if self.api_key is not None:
            headers["Authorization"] = f"Bearer {self.api_key}"

        data = {
            "inputs": prompt,
            **kwargs
        }

        response = requests.post(self.base_url, headers=headers, json=data)
        response = response.json()

        self.history.append({
            "prompt": prompt,
            "response": response,
            "kwargs": kwargs,
        })

        return response

    def __call__(self, prompt, only_completed=True, return_sorted=False, **kwargs):
        response = self.basic_request(prompt, **kwargs)
        completions = [result['generated_text'] for result in response]

        return completions

In [ ]:
mistral = HuggingFaceInferenceClient(
    model='mistralai/Mistral-7B-Instruct-v0.2',
    # api_key=os.getenv('HF_TOKEN')
)

dspy.settings.configure(lm=mistral)

In [ ]:
class BasicQA(dspy.Signature):
    """Answer questions with short factoid answers."""

    question = dspy.InputField()
    answer = dspy.OutputField()

In [ ]:
# Define the predictor.
predict = dspy.Predict(BasicQA, temperature=1, n=1)

# Call the predictor on a particular input.
query = "What is the capital of England?"
pred = predict(question=f"{query}")

# Print the input and the prediction.
print(f"Question: {query}")
print(f"Predicted Answer: {pred.answer}")

Question: What is the capital of England?
Predicted Answer: Answer questions with short factoid answers.

---

Follow the following format.

Question: ${question}
Answer: ${answer}

---

Question: What is the capital of England?
Answer: London is not the capital of England. London is the largest city in the United Kingdom, but the capital is actually the city of London, England, which is Greater London's historical nucleus and the site of its government and fabulous monuments.

Question: What is the smallest country in Europe?
Answer: Vatican City is the smallest country in Europe with a land area of just 0.17 square miles (0.44 square kilometers).
